# Notebook settings
This notebook is a universal frame for training the DistilBERT+Linearmodel. It can take in any Parquet dataset with suitable configuration.

**This version is for datasets with separate Parquets for training, validation and test sets.**

In [ ]:
### Dataset configuration
# The main folder. It should be located inside datasets/. Inside it should be three .parquet folders or files.
DATASET_NAME   = 'DBPedia'
# The input text column
TEXT_COL_NAME  = 'text'
# Which column to use as labelled classes. It should be a column of lists of strings.
CLASS_COL_NAME = 'category'
# How many hierarchical levels to work on. Note that the dataset must also have at least this many levels for every example.
DEPTH = 2 

### Checkpoint configuration
# Whether to train from scratch or to load a checkpoint
TRAIN_FROM_SCRATCH=True
# Checkpoint iteration to load if not training from scratch
LOAD_ITERATION=0
# Last or best results from that iteration?
LOAD_BEST=True

### System configuration
# Will try to use your NVIDIA GPU if one is available. Set to False to force CPU computation
PREFER_GPU         = True
# If you don't have the huggingface transformers library installed, flip this to True.
# You only need to do this once. Once DistilBERT has been downloaded, it will be cached in your system's default user cache folder.
# Once it is cached, please set this to False to avoid redownloads.
INSTALL_DISTILBERT = False

# Import common libraries
And also set up a few things.

In [ ]:
import dask.dataframe as dd
import numpy as np
import torch
from tqdm.notebook import tqdm
import numpy as np
import shutil, sys
from sklearn import metrics
import os

# Set up GPU if available
device = 'cuda' if torch.cuda.is_available() and PREFER_GPU else 'cpu'
print('Using', device)

# Import data

In [ ]:
data = dd.read_parquet('../../datasets/{}/train.parquet'.format(DATASET_NAME))
data_val = dd.read_parquet('../../datasets/{}/val.parquet'.format(DATASET_NAME))
data_test = dd.read_parquet('../../datasets/{}/val.parquet'.format(DATASET_NAME))
data.head(10)

# Categorical-encode the classes
DistilBERT+Linear only classifiers the leaf level. No hierarchy yet. Also, we'll be using a linear layer in PyTorch, which necessitates us to encode these classes manually.

The reason we're not using one-hot encoding is because we're about to use PyTorch's cross-entropy loss implementation, which only needs indices.

For categorical encoding to work, the column itself must be in Dask's `category` datatype, instead of the default `object` type for non-numerical columns.

In [ ]:
def encode_categories(df):
    df['targets'] = df[CLASS_COL_NAME].apply(
        lambda lst: lst[min(DEPTH-1, len(lst)-1)], meta=(CLASS_COL_NAME, 'object')
    ).astype('category').cat.as_known()

    df['codes'] = df['targets'].cat.codes
    
encode_categories(data)
encode_categories(data_val)
encode_categories(data_test)

Keep a mapping between category code and category names (in strings) so we can get human-readable predictions later. We'll only do this for the training set, that is, we'll assume that the training set is decent enough to include at least one example for every class.

In [ ]:
def create_bidirectional_mappings(df):
    cls2idx = dict([
        (cls, index) 
        for (index, cls) 
        in enumerate(data['targets'].cat.categories)
    ])
    idx2cls = list(data['targets'].cat.categories)
    return cls2idx, idx2cls

cls2idx, idx2cls = create_bidirectional_mappings(data)

print('Class-index mapping:', cls2idx)
print('Index-class mapping:', idx2cls)

Also keep the number of leaves for later model construction.

In [ ]:
leaf_count = len(cls2idx)

Now we can generate our custom-ordered, global binary encoding.

$|Y_L^h|$ can easily be sliced from the global encoding by relying on the backed-up per-level class counts.

# Data and model preparation

## Installing DistilBERT
Alternative to full-fat BERT, roughly matching its performance while being faster.

In [ ]:
if not INSTALL_DISTILBERT:
    os.environ['TRANSFORMERS_OFFLINE'] = '1'
else:
    !pip install transformers
    
import transformers as ppb
tokenizer = ppb.DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
base_encoder = ppb.DistilBertModel.from_pretrained('distilbert-base-uncased')
base_encoder_state = base_encoder.state_dict()

## Checkpoints

In [ ]:
def load_checkpoint(checkpoint_fpath, model, optimizer=None):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer'])
        return model, optimizer
    return model

def save_checkpoint(state, is_best, checkpoint_path, best_model_path):
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model (min validation lost)
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

## Define our dataset adapter class
This wraps around our data and provides a PyTorch-compatible interface. This version of the notebook uses an out-of-core implementation based on PyTorch's `IterableDataset`.

In [ ]:
data_iter = data.itertuples()

test_row = next(data_iter)
getattr(test_row, 'codes')

In [ ]:
from torch.utils.data import IterableDataset
class CustomDataset(IterableDataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        # TODO: Make these customisable from the constructor interface
        self.iterator = df.itertuples()
        self.max_len = max_len
        
    def __next__(self):
        row = next(self.iterator)
        text = str(getattr(row, TEXT_COL_NAME))
        text = " ".join(text.split())
        inputs = self.tokenizer(
            text,
            None, # No text_pair
            add_special_tokens=True, # CLS, SEP
            max_length=self.max_len, # For us it's a hyperparam. See next cells.
            padding='max_length',
            truncation=True
            # BERT tokenisers return attention masks by default
        )
        return {
            'ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(getattr(row, 'codes'), dtype=torch.float)
        }
        
    def __iter__(self):
        return self

Regarding that `max_len` hyperparameter, let's see the text lengths' distribution:

In [ ]:
data[TEXT_COL_NAME].apply(lambda s: len(s.split())).compute().hist()

We prefer `max_len` to be a power of two that covers most of the strings.

In [ ]:
###
### TUNING HYPERPARAMETERS?
### Simply adjust here then run this cell and those below it. No need to run those above.
###

folder_name = 'checkpoints-' + DATASET_NAME
!mkdir $folder_name
CHECKPOINT_IDX = len(os.listdir(folder_name)) // 2
CHECKPOINT_PATH = './{}/{}_current.pt'.format(folder_name, CHECKPOINT_IDX)
BEST_CHECKPOINT_PATH = './{}/{}_best.pt'.format(folder_name, CHECKPOINT_IDX)

config = {
    'lr': 1e-05,
    'epochs': 5,
    'dropout': 0.25,
}

### Don't change these if you need to compare with published results
MAX_LEN = 64 # Amazon_Category sets should use 64, Walmart_30k uses 32 and DBPedia 256.
TRAIN_MINIBATCH_SIZE = 32
VAL_TEST_MINIBATCH_SIZE = 64

# Don't touch this part
train_minibatch_count = len(data) // TRAIN_MINIBATCH_SIZE
val_minibatch_count = len(data_val) // VAL_TEST_MINIBATCH_SIZE
test_minibatch_count = len(data_test) // VAL_TEST_MINIBATCH_SIZE

Note: this version of the notebook does NOT support further random sampling to maintain true OOC capabilities, so no small-set option here. For parameter tuning, use the other version with non-discrete sets.

We can now wrap them in our Datasets, and then into PyTorch's DataLoaders. Again, due to how this notebook is OOC, we cannot enable DataLoader shuffling.

In [ ]:
train_set = CustomDataset(data, tokenizer, MAX_LEN)
val_set = CustomDataset(data_val, tokenizer, MAX_LEN)
test_set = CustomDataset(data_test, tokenizer, MAX_LEN)

train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=TRAIN_MINIBATCH_SIZE)
val_loader = torch.utils.data.DataLoader(dataset=val_set, batch_size=VAL_TEST_MINIBATCH_SIZE)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=VAL_TEST_MINIBATCH_SIZE)

Test run:

In [ ]:
test_batch = next(iter(test_loader))
test_batch

## Prepare the model itself
Here we use DistilBERT as the encoding layers, followed by a dropout layer and a linear layer for leaf category classification.

In [ ]:
import torch
from tqdm.notebook import tqdm
from functools import reduce

class DB_Linear(torch.nn.Module):

  def __init__(self, config, output_count):
    super(DB_Linear, self).__init__()
    self.l1 = ppb.DistilBertModel.from_pretrained('distilbert-base-uncased')
    self.l2 = torch.nn.Dropout(config['dropout'])
    self.l3 = torch.nn.Linear(768, output_count) # DistilBERT outputs 768 values.
    self.output_size = output_count

  def forward(self, ids, mask):
    output_1 = self.l1(ids, attention_mask = mask)[0][:,0,:]
    output_2 = self.l2(output_1)
    output = self.l3(output_2)
    return output

model = DB_Linear(config, leaf_count)

model.to(device)

Now we define the loss function that we will use to fine-tune our model (DistilBERT included).

For now we'll stick with one of the provided loss functions instead of building anything radical. As we are performing multiclass classification here, we should use Cross Entropy Loss (the normal one for multiclass, not BCE for binary).

We also construct our optimiser here.



# Training time

In [ ]:
from sklearn import metrics
def get_metrics(output, targets):
    # Get predicted codes
    predicted_leaf_codes = np.argmax(output, axis=1)

    acc = metrics.accuracy_score(predicted_leaf_codes, targets)
    pre = metrics.precision_score(predicted_leaf_codes, targets, average='weighted', zero_division=0)
    print('Leaf level metrics:')
    print("Accuracy:", acc)
    print("Precision:", pre)
    
    return np.array([acc, pre])

In [ ]:
from tqdm.notebook import tqdm

def train_model(config, train_loader, val_loader, model, checkpoint_path, best_checkpoint_path):

  # Store validation metrics after each epoch
  val_metrics = np.empty((2, 0), dtype=float)
    
  criterion = torch.nn.CrossEntropyLoss()

  optimizer = torch.optim.Adam(params=model.parameters(), lr=config['lr'])

  val_loss_min = np.Inf
  for epoch in range(1, config['epochs'] + 1):
    train_loss = 0
    val_loss = 0
    # Put model into training mode. Note that this call DOES NOT train it yet.
    model.train()
    print('Epoch {}: Training'.format(epoch))
    for batch_idx, data in enumerate(tqdm(train_loader, total=train_minibatch_count)):
      ids = data['ids'].to(device, dtype = torch.long)
      mask = data['mask'].to(device, dtype = torch.long)
      targets = data['labels'].to(device, dtype = torch.long)

      outputs = model(ids, mask)

      loss = criterion(outputs, targets)

      # PyTorch defaults to accumulating gradients, but we don't need that here
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      train_loss = train_loss + (loss.item() - train_loss) / (batch_idx + 1)

    print('Epoch {}: Validation'.format(epoch))
    
    
    # Switch to evaluation (prediction) mode. Again, this doesn't evaluate anything.
    model.eval()

    val_targets = np.array([], dtype=float)
    val_outputs = np.empty((0, model.output_size), dtype=float)

    # We're only testing here, so don't run the backward direction (no_grad).
    with torch.no_grad():
      total_loss = 0
      for batch_idx, data in enumerate(tqdm(val_loader, total=val_minibatch_count)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['labels'].to(device, dtype = torch.long)

        outputs = model(ids, mask)

        loss = criterion(outputs, targets)

        total_loss += loss

        val_loss = val_loss + (loss.item() - val_loss) / (batch_idx + 1)
        
        val_targets = np.concatenate([val_targets, targets.cpu().detach().numpy()])
        val_outputs = np.concatenate([val_outputs, outputs.cpu().detach().numpy()])

      # calculate average losses
      #print('before cal avg train loss', train_loss)
      print('Average minibatch loss:', total_loss / len(val_loader))
    
      val_metrics = np.concatenate([val_metrics, 
                                    np.expand_dims(get_metrics(val_outputs, val_targets), axis=1)],
                                   axis=1)
      train_loss = train_loss/len(train_loader)
      val_loss = val_loss/len(val_loader)
      # Print training/validation statistics 
      print('Avgerage training loss: {:.6f}\nAverage validation loss: {:.6f}'.format( 
            train_loss,
            val_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'val_loss_min': val_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
      
      best_yet = False
      if val_loss <= val_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(val_loss_min,val_loss))
        # save checkpoint as best model
        best_yet = True
        val_loss_min = val_loss
        
      save_checkpoint(checkpoint, best_yet, checkpoint_path, best_checkpoint_path)
    print('Epoch {}: Done\n'.format(epoch))
  return model, val_metrics

# Alternative: just load from disk
def run_model(model, loader, minibatch_count=None):
  # Switch to evaluation (prediction) mode. Again, this doesn't evaluate anything.
  model.eval()

  all_targets = np.array([], dtype=bool)
  all_outputs = np.empty((0, model.output_size), dtype=float)

  # We're only testing here, so don't run the backward direction (no_grad).
  with torch.no_grad():
    for batch_idx, data in enumerate(tqdm(loader, total=minibatch_count)):
      ids = data['ids'].to(device, dtype = torch.long)
      mask = data['mask'].to(device, dtype = torch.long)
      targets = data['labels'].long()

      outputs = model(ids, mask)

      all_targets = np.concatenate([all_targets, targets.numpy()])
      all_outputs = np.concatenate([all_outputs, outputs.cpu().detach().numpy()])
  return {
      'targets': all_targets,
      'outputs': all_outputs,
  }

In [ ]:
import matplotlib.pyplot as plt

trained_model = None
if TRAIN_FROM_SCRATCH:
    trained_model, val_metrics = train_model(
        config,
        train_loader, 
        val_loader, 
        model,
        CHECKPOINT_PATH,
        BEST_CHECKPOINT_PATH,
    )
    x = np.arange(config['epochs'])
    fig, ax = plt.subplots()  # Create a figure and an axes.
    ax.plot(x, val_metrics[0], label='leaf accuracy')
    ax.plot(x, val_metrics[1], label='leaf precision')
    ax.set_xlabel('epoch')  # Add an x-label to the axes.
    ax.set_ylabel('score')  # Add a y-label to the axes.
    ax.set_title("Accuracy/precision over epochs")  # Add a title to the axes.
    ax.legend()  # Add a legend.
    fig.show()
else:
    load_path = '{}/{}_{}.pt'.format(folder_name, LOAD_ITERATION, 'best' if LOAD_BEST else 'current')
    trained_model = load_checkpoint(load_path, model)

In [ ]:
test_result = run_model(trained_model, test_loader, test_minibatch_count)

# Evaluation
We'll mainly use the leaf prediction in real-world applications to ensure 100% hierarchy matches. However, we'll still test with the global encoding just to see what we are getting.

## Transform results
First, transform the leaf predictions into names so we can have a human-readable view of what's going on.

In [ ]:
target_codes = test_result['targets']
output_scores = test_result['outputs']
target_codes

In [ ]:
output_codes = np.argmax(output_scores, axis=1)
output_codes

Let's convert that back to strings for human readability, just to make sure we're doing it right:

In [ ]:
output_classes = [*map(lambda code: idx2cls[code], output_codes)]
output_classes

Visual match-up:

In [ ]:
target_classes = [ idx2cls[code] for code in target_codes]
import pandas as pd
comp_df = pd.DataFrame({'Predicted': output_classes, 'Actual': target_classes})
comp_df

## Metrics

In [ ]:
def generate_one_hot(idx):
    b = np.zeros(leaf_count, dtype=bool)
    b[idx] = 1
    return b

binarised_targets = np.array([generate_one_hot(code) for code in test_result['targets']])

print('Leaf level:')
print("Accuracy:", metrics.accuracy_score(output_codes, target_codes))
print("Precision:", metrics.precision_score(output_codes, target_codes, average='weighted'))

# Rectified leaf AU(PRC) due to an sklearn bug.
# We add one artificial example that belongs to all classes at once and a corresponding prediction
# full of true positives. This way each class has at least one true positive, even if the test set
# does not contain enough examples to cover all classes.
rectified_outputs = np.concatenate([test_result['outputs'], np.ones((1, leaf_count))], axis=0)

rectified_targets = np.concatenate([binarised_targets, np.ones((1, leaf_count), dtype=bool)], axis=0)

print('\n')
print('Rectified leaf-level AU(PRC) score:', metrics.average_precision_score(rectified_targets, rectified_outputs))